In [7]:
from datetime import *
import time
from dotenv import load_dotenv
import os, requests, json
from pymongo import MongoClient

In [8]:
API_KEY = os.getenv('API_KEY')

ADD CORRECT DATABASE!

In [9]:
client = MongoClient('mongodb://localhost:27017')
db = client.artblocks
db.art

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'artblocks'), 'art')

MUST CHANGE DATES!!!

In [10]:
# Given a date range return values of each day start and end in between the range 
# Last Page: cj0xJnA9MjAyMi0wMi0wMiswMiUzQTQ1JTNBMTIuNjQ3MDM2

start_date = datetime(2020, 11, 1, 0, 0)
end_date = datetime(2022, 6, 10, 0, 0)
times = []

while start_date != end_date:
    start_date += timedelta(days=1)
    times.append(start_date.timestamp()) # Returns Epoch time for 00:00 am EST


In [11]:
def get_events(cursor, after, eventType='offer_entered', assetContract = '0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270', **kwargs):
    url = f"https://api.opensea.io/api/v1/events?asset_contract_address={assetContract}&event_type={eventType}&occurred_after={after}&cursor={cursor}"

    headers = {
    "Accept": "application/json",
    "X-API-KEY": API_KEY
    }

    response = requests.get(url, headers=headers).text
    response = json.loads(response)
    return response

WRITE TO CORRECT DATABASE!!!

In [12]:
last = 'cj0xJnA9MjAyMi0wMi0wMiswMiUzQTQ1JTNBMTIuNjQ3MDM2'
prev = ''

response = get_events(last, times[0])
print(response['previous'])

for t in times:
    i = 0
    response = get_events(last, t)
    prev = response['previous']
    db.art.insert_one(response)
    time.sleep(.33)

    while (i < 10):
        i += 1
        response = get_events(prev, t)
        db.art.insert_one(response)
        time.sleep(.33)

    print(f'Time {t} completed')

None


TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping